### Sample size effect in gene co-expression networks (with graph-tool)

- Author: Joaquim Aguirre Plans<br>
- Institution: Northeastern University

Load libraries:

In [1]:
import sys, os
import graph_tool.all as gt
import matplotlib.pyplot as plt
import multiprocessing as mp
import networkx as nx
import numpy as np
import pandas as pd
import pylab
import random
import seaborn as sns
import scipy
import time
import NetworkMedicineToolbox.wrappers as wrappers
import NetworkMedicineToolbox.network_utilities as network_utilities
import coexpression_analysis_functions_graphtool as cagt
import coexpression_analysis_functions_networkx as canx
random.seed(1510)

/home/j.aguirreplans/.conda/envs/ScipherSampleSizeEnv/lib/python3.7/site-packages/graph_tool/draw/cairo_draw.py:1494: RuntimeWarning: Error importing Gtk module: No module named 'gi'; GTK+ drawing will not work.
  warnings.warn(msg, RuntimeWarning)


GenRev not found, steiner wont work
Import error: Negex. Using keyword matching instead
Import error: Funcassociate. Make sure that funcassociate is in toolbox!
DIAMOnD not found and thus will not be available!


## Sample size effect in network size and composition

In this part, I check the effect of the sample size in the size of gene co-expression networks.

In [2]:
# Define directories and files
scratch_data_dir = '/scratch/j.aguirreplans/Scipher/SampleSize'
home_data_dir = '/home/j.aguirreplans/Projects/Scipher/SampleSize/data'
ppi_dir = '/home/j.aguirreplans/data/PPI'
wto_results_dir = os.path.join(scratch_data_dir, 'networks_nonresponders_wto_N100')
#wto_results_dir = os.path.join(scratch_data_dir, 'example_networks_nonresponders_wto')
output_dir = os.path.join(home_data_dir, 'out')
plots_dir = os.path.join(output_dir, 'plots')
#plots_dir = os.path.join(output_dir, 'plots_example')


In [3]:
# Define parameters of bootstrap
rep = 10
step = 10
max_num_samples = 56
size_list = list(range(10, max_num_samples+1, step))
print('List of sample sizes considered: {}'.format(', '.join([str(size) for size in size_list])))
print('Number of repetitions of each sample size: {}'.format(rep))

List of sample sizes considered: 10, 20, 30, 40, 50
Number of repetitions of each sample size: 10


In [31]:
# Parse global network using pandas
start = time.time() # Starting time
pval_adj_cutoff = 0.05
global_network_df = pd.read_csv(global_network_file)
global_network_df = network_df[network_df["pval.adj"] < pval_adj_cutoff]
end = time.time() # End time
print("Runtime for loading the network is {}".format(end-start))


Runtime for loading the network is 61.561196088790894


In [32]:
global_network_df

,Node.1,Node.2,wTO,pval,pval.adj
0,5S_rRNA,7SK,0.091,0.00,0.000000
5,A1BG,A1BG-AS1,0.363,0.01,0.038676
6,5S_rRNA,A2M,0.107,0.01,0.038676
7,7SK,A2M,0.067,0.01,0.038676
10,5S_rRNA,A2M-AS1,0.154,0.00,0.000000
...,...,...,...,...,...
115390827,ZW10,ZZZ3,0.620,0.00,0.000000
115390828,ZWILCH,ZZZ3,0.641,0.00,0.000000
115390829,ZWINT,ZZZ3,0.548,0.01,0.038676
115390830,ZXDA,ZZZ3,0.633,0.00,0.000000


In [33]:
# Sort the edges of the network
start = time.time() # Starting time
#global_network_sorted_df = global_network_df[["Node.1", "Node.2"]].apply(lambda x: sorted(x), axis=1)
global_network_df["both"] = global_network_df[["Node.1", "Node.2"]].values.tolist()
end = time.time() # End time
print("Runtime for sorting the network is {}".format(end-start))


Runtime for sorting the network is 31.155787706375122


/home/j.aguirreplans/.conda/envs/ScipherSampleSizeEnv/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [49]:
global_network_df[10000000:10000020]

,Node.1,Node.2,wTO,pval,pval.adj,both
41147046,APMAP,NTNG2,0.196,0.00,0.000000,"[APMAP, NTNG2]"
41147049,APOBEC3A,NTNG2,0.192,0.01,0.038676,"[APOBEC3A, NTNG2]"
41147057,APOL1,NTNG2,0.210,0.01,0.038676,"[APOL1, NTNG2]"
41147058,APOL2,NTNG2,0.222,0.01,0.038676,"[APOL2, NTNG2]"
41147072,AQP1,NTNG2,0.172,0.01,0.038676,"[AQP1, NTNG2]"
41147075,AQP9,NTNG2,0.055,0.01,0.038676,"[AQP9, NTNG2]"
41147084,ARF1,NTNG2,0.307,0.00,0.000000,"[ARF1, NTNG2]"
41147103,ARHGAP11B,NTNG2,-0.180,0.01,0.038676,"[ARHGAP11B, NTNG2]"
41147116,ARHGAP27,NTNG2,0.361,0.00,0.000000,"[ARHGAP27, NTNG2]"
41147125,ARHGAP44,NTNG2,0.139,0.01,0.038676,"[ARHGAP44, NTNG2]"


In [35]:
# Sort the edges of the network
start = time.time() # Starting time
global_network_edges = global_network_df["both"].apply(lambda x: sorted(x))
end = time.time() # End time
print("Runtime for sorting the network is {}".format(end-start))


Runtime for sorting the network is 29.31928300857544


In [50]:
global_network_edges[10000000:10000020]


41147046        [APMAP, NTNG2]
41147049     [APOBEC3A, NTNG2]
41147057        [APOL1, NTNG2]
41147058        [APOL2, NTNG2]
41147072         [AQP1, NTNG2]
41147075         [AQP9, NTNG2]
41147084         [ARF1, NTNG2]
41147103    [ARHGAP11B, NTNG2]
41147116     [ARHGAP27, NTNG2]
41147125     [ARHGAP44, NTNG2]
41147128      [ARHGAP9, NTNG2]
41147153       [ARID3A, NTNG2]
41147184        [ARL8A, NTNG2]
41147189        [ARMC2, NTNG2]
41147207       [ARPC1B, NTNG2]
41147210        [ARPC4, NTNG2]
41147216        [ARRB2, NTNG2]
41147223         [ARSA, NTNG2]
41147248        [ASF1B, NTNG2]
41147260       [ASPHD2, NTNG2]
Name: both, dtype: object

In [5]:
# Create network object (using GT)
start = time.time() # Starting time
network = gt.Graph(directed=False)
network_ids = network.add_edge_list(global_network_df[["Node.1", "Node.2"]].values, hashed=True)
end = time.time() # End time
print("Runtime for loading the network into Graph-Tool is {}".format(end-start))
print('Global gene co-expression network: {} nodes and {} edges'.format(len(global_sig_network.get_vertices()), len(global_sig_network.get_edges())))


Runtime for loading the network is 141.1483850479126
Global gene co-expression network: 15192 nodes and 29835534 edges


In [ ]:
df = pd.DataFrame({"s": [], "t": []})

In [21]:
start = time.time() # Starting time
global_sig_network_nodes = set(global_sig_ids)
end = time.time() # End time
print("Runtime for loading the nodes is {}".format(end-start))
start = time.time() # Starting time
global_sig_network_edges = [sorted([global_sig_ids[u], global_sig_ids[v]]) for u,v in global_sig_network.get_edges()]
end = time.time() # End time
print("Runtime for loading the edges is {}".format(end-start))
print('Global gene co-expression network: {} nodes and {} edges'.format(len(global_sig_network_nodes), len(global_sig_network_edges)))


Runtime for loading the nodes is 0.045685529708862305
Runtime for loading the edges is 234.03956198692322
Global gene co-expression network: 15192 nodes and 29835534 edges


In [7]:
# Parse global network (using NX)
global_network_file = os.path.join(wto_results_dir, 'wto_RNAseq_NonResponders_all.net')

start = time.time() # Starting time
global_sig_network_nx = canx.parse_significant_coexpression_network_from_wto(global_network_file, pval_adj_cutoff=0.05)
end = time.time() # End time
print("Runtime for loading the network is {}".format(end-start))

print('Global gene co-expression network: {} nodes and {} edges'.format(len(global_sig_network.nodes()), len(global_sig_network.edges())))
#global_sig_network_nodes = set(global_sig_network_nx.nodes())
#global_sig_network_edges = set(global_sig_network_nx.edges())


Runtime for loading the network is 166.07460260391235
Global gene co-expression network: 15192 nodes and 29835534 edges


In [ ]:
# Parse PPI network
#ppi_network_file = os.path.join(ppi_dir, 'interactome_2019_merged.csv')
#ppi_info_file = os.path.join(ppi_dir, 'interactome_2019_merged_protAnnots.csv')
#ppi_info = pd.read_csv(ppi_info_file, index_col=0, header=0) # Read PPI info
#ppi = parse_ppi_network_with_networkx(ppi_network_file)
#print('PPI network: {} nodes and {} edges'.format(len(ppi.nodes()), len(ppi.edges())))
#ppi = parse_ppi_network_with_graphtools(ppi_network_file)
#print('PPI network: {} nodes and {} edges'.format(len(ppi.get_vertices()), len(ppi.get_edges())))


In [7]:
# Parse disease genes associated to diseases with 20 genes or more
disease_genes_file = '/home/j.aguirreplans/Projects/Scipher/SampleSize/data/Guney2016_GenesDisease.tsv'
disease2genes = {}
for i in open(disease_genes_file).readlines():
    v = i.rstrip().split('\t')
    disease = v[1]
    genes = v[2:]
    if len(genes) > 19:
        disease2genes[disease] = [int(i) for i in genes]

In [11]:
diseasegenes = list(disease2genes.values())
diseasegenes = sum(diseasegenes, [])
diseasegenes = list(set(diseasegenes))
len(diseasegenes)

3173

In [13]:
# Parse RA genes
ra_file = os.path.join(home_data_dir, 'RA_Genes.txt')
RA_genes = list(pd.read_csv(ra_file, sep=' ', header=None).iloc[:, 0])
RA_genes_network = [gene for gene in RA_genes if gene in global_sig_network_nodes]
print('RA: {} genes, {} genes in gene co-expression network'.format(len(RA_genes), len(RA_genes_network)))
#print(RA_genes_network)
print(len(disease2genes['arthritis, rheumatoid']))


RA: 447 genes, 0 genes in gene co-expression network
51


In [ ]:
# Parse subsample networks
size_df = pd.DataFrame(columns=['size', 'rep', 'nodes', 'edges'])
composition_df = pd.DataFrame(columns=['size', 'rep', 'nodes', 'edges', 'lost_or_gained'])
kendall_df = pd.DataFrame(columns=['size', 'kendall'])
components_df = pd.DataFrame(columns=['size', 'rep', 'num_components', 'size_lcc'])
RA_df = pd.DataFrame(columns=['size', 'rep', 'RA_genes_in_network', 'RA_lcc_size'])
for size in size_list:
    #wto_list_of_same_sample_size = []
    for r in range(1, rep+1):
        subsample_network_file = os.path.join(wto_results_dir, 'wto_RNAseq_NonResponders_sample_{}_rep_{}.net'.format(size, r))
        if fileExist(subsample_network_file):


            # Parse subsample network
            start = time.time() # Starting time
            #subsample_network = parse_coexpression_network_from_wto_with_networkx(subsample_network_file)
            subsample_sig_network = parse_significant_coexpression_network_from_wto_with_networkx(subsample_network_file, pval_adj_cutoff=pval_adj_cutoff)
            print('THE NETWORK HAS {} EDGES AND {} NODES'.format(len(subsample_sig_network.edges()), len(subsample_sig_network.nodes())))

            # Get weights & ranks associated to weights
            #wto_weights = [subsample_network[u][v]['wto'] for u,v in global_network.edges()]
            #wto_ranks = scipy.stats.rankdata(wto_weights)
            #wto_list_of_same_sample_size.append(wto_ranks)
            
            # Get significant network
            #subsample_sig_network = filter_network_by_pval_adj_with_networkx(subsample_network, pval_adj_cutoff)
            #print('THE NETWORK HAS {} EDGES AND {} NODES'.format(len(subsample_sig_network.edges()), len(subsample_sig_network.nodes())))
            df2 = pd.DataFrame([[size, r, len(subsample_sig_network.nodes()), len(subsample_sig_network.edges())]], columns=['size', 'rep', 'nodes', 'edges'])
            #subsample_network, subsample_network_wto_prop, subsample_network_pval_adj_prop = parse_coexpression_network_from_wto_with_graphtools(subsample_network_file)
            #subsample_sig_network = filter_network_by_pval_adj_with_graphtools(subsample_network, subsample_network_pval_adj_prop, pval_adj_cutoff)
            #print('Subsample gene co-expression network: {} nodes and {} edges'.format(len(subsample_sig_network.get_vertices()), len(subsample_sig_network.get_edges())))
            #df2 = pd.DataFrame([[size, r, len(subsample_sig_network.get_vertices()), len(subsample_sig_network.get_edges())]], columns=['size', 'rep', 'nodes', 'edges'])
            size_df = size_df.append(df2)
            end = time.time() # End time
            print("Runtime for loading the network in sample size {} and rep {} is {}".format(size, r, end-start))

            # Get components and LCC
            start = time.time() # Starting time
            subsample_sig_components = network_utilities.get_connected_components(G=subsample_sig_network, return_as_graph_list=False)
            subsample_sig_lcc = subsample_sig_network.subgraph(subsample_sig_components[0])
            df2 = pd.DataFrame([[size, r, len(subsample_sig_components), len(subsample_sig_lcc.nodes())]], columns=['size', 'rep', 'num_components', 'size_lcc'])
            components_df = components_df.append(df2)
            subsample_sig_components = []
            subsample_sig_lcc = []
            end = time.time() # End time
            print("Runtime for calculating the components in sample size {} and rep {} is {}".format(size, r, end-start))

            # Get connected RA genes
            start = time.time() # Starting time
            RA_subgraph = subsample_sig_network.subgraph(RA_genes)
            RA_components = network_utilities.get_connected_components(G=RA_subgraph, return_as_graph_list=False)
            RA_lcc = RA_subgraph.subgraph(RA_components[0])
            df2 = pd.DataFrame([[size, r, len(RA_subgraph.nodes()), len(RA_lcc.nodes())]], columns=['size', 'rep', 'RA_genes_in_network', 'RA_lcc_size'])
            RA_df = RA_df.append(df2)
            RA_subgraph = []
            RA_components = []
            RA_lcc = []
            end = time.time() # End time
            print("Runtime for calculating the RA genes in sample size {} and rep {} is {}".format(size, r, end-start))

            # Calculate differences between global and subsample network
            start = time.time() # Starting time
            subsample_sig_network_edges = set(subsample_sig_network.edges())
            end = time.time() # End time
            print("Runtime for calculating the set of edges in sample size {} and rep {} is {}".format(size, r, end-start))
            start = time.time() # Starting time
            subsample_sig_network_nodes = set(subsample_sig_network.nodes())
            end = time.time() # End time
            print("Runtime for calculating the set of nodes in sample size {} and rep {} is {}".format(size, r, end-start))
            start = time.time() # Starting time
            #lost_nodes, gained_nodes, lost_edges, gained_edges = calculate_difference_between_networks_with_networkx(global_sig_network, subsample_sig_network)
            lost_nodes, gained_nodes, lost_edges, gained_edges = calculate_difference_between_networks_with_networkx(global_sig_network_nodes, global_sig_network_edges, subsample_sig_network_nodes, subsample_sig_network_edges)
            #print(size, r, len(lost_edges), len(gained_edges))
            df3 = pd.DataFrame([[size, r, len(lost_nodes), len(lost_edges), 'lost']], columns=['size', 'rep', 'nodes', 'edges', 'lost_or_gained'])
            composition_df = composition_df.append(df3)
            #lost_nodes, gained_nodes, lost_edges, gained_edges = calculate_difference_between_networks_with_graphtools(global_sig_network, subsample_sig_network)
            #df3 = pd.DataFrame([[size, r, len(lost_nodes), len(lost_edges), 'lost']], columns=['size', 'rep', 'nodes', 'edges', 'lost_or_gained'])
            #composition_df = composition_df.append(df3)
            df4 = pd.DataFrame([[size, r, len(gained_nodes), len(gained_edges), 'gained']], columns=['size', 'rep', 'nodes', 'edges', 'lost_or_gained'])
            composition_df = composition_df.append(df4)
            subsample_sig_network_edges = []
            subsample_sig_network_nodes = []
            lost_nodes = []
            lost_edges = []
            gained_nodes = []
            gained_edges = []
            end = time.time() # End time
            print("Runtime for calculating the composition in sample size {} and rep {} is {}".format(size, r, end-start))

        else:
            print('Missing file: {}'.format(subsample_network_file))
        #break
    #W, count = calculate_kendall(np.array(wto_list_of_same_sample_size), n_permutations=1000)
    #df5 = pd.DataFrame([[size, W]], columns=['size', 'kendall'])
    #kendall_df = kendall_df.append(df5)
    #break
#print(size_df)
#print(composition_df)
#print(components_df)
#print(RA_df)


### Sample size effect on network size

I plot the number of nodes and edges of the different networks with respect to the sample size.

In [ ]:
# Plot num. of nodes vs sample size
fig = pylab.figure(dpi=300)
sns.boxplot(data=size_df, x='size', y='nodes', color='dodgerblue')
plt.title('Number of nodes vs. sample size')
plt.xlabel('Sample size')
plt.ylabel('Number of nodes')
pylab.savefig(os.path.join(plots_dir, 'num_nodes_vs_sample_size.png'), format='png')
plt.show()

# Plot num. of edges vs sample size
fig = pylab.figure(dpi=300)
sns.boxplot(data=size_df, x='size', y='edges', color='dodgerblue')
plt.title('Number of edges vs. sample size')
plt.xlabel('Sample size')
plt.ylabel('Number of edges')
pylab.savefig(os.path.join(plots_dir, 'num_edges_vs_sample_size.png'), format='png')
plt.show()

### Sample size effect in network composition

Here, I check the impact of sample size in the composition of nodes and edges of gene co-expression networks. With this purpose, I check the number of lost and gained edges with respect to the network that has the 100% of samples.

In [ ]:
# Plot num. of nodes lost and gained vs sample size
fig = pylab.figure(dpi=300)
sns.boxplot(data=composition_df, x="size", y="nodes", hue="lost_or_gained", palette=["dodgerblue", "gold"])
plt.title('Number of lost and gained nodes vs. sample size')
plt.xlabel('Sample size')
plt.ylabel('Number of nodes')
pylab.savefig(os.path.join(plots_dir, 'num_lost_nodes_vs_sample_size.png'), format='png')
plt.show()

# Plot num. of edges lost and gained vs sample size
fig = pylab.figure(dpi=300)
sns.boxplot(data=composition_df, x="size", y="edges", hue="lost_or_gained", palette=["dodgerblue", "gold"])
plt.title('Number of lost and gained edges vs. sample size')
plt.xlabel('Sample size')
plt.ylabel('Number of edges')
pylab.savefig(os.path.join(plots_dir, 'num_lost_edges_vs_sample_size.png'), format='png')
plt.show()

### Sample size effect on the number of components and size of the LCC

In [ ]:
# Plot num. of components vs. sample size
fig = pylab.figure(dpi=300)
sns.boxplot(data=components_df, x='size', y='num_components', color='dodgerblue')
plt.title('Number of connected components in the gene co-expression network')
plt.xlabel('Sample size')
plt.ylabel('Number of connected components')
pylab.savefig(os.path.join(plots_dir, 'num_connected_components_vs_sample_size.png'), format='png')
plt.show()


In [ ]:
# Plot num. of components vs. sample size
fig = pylab.figure(dpi=300)
sns.boxplot(data=components_df, x='size', y='size_lcc', color='dodgerblue')
plt.title('Size of the LCC in the gene co-expression network')
plt.xlabel('Sample size')
plt.ylabel('Number of nodes in the LCC')
pylab.savefig(os.path.join(plots_dir, 'LCC_size_vs_sample_size.png'), format='png')
plt.show()


## Sample size effect on reproducibility of the network

In [ ]:
#fig = pylab.figure(dpi=300)
#sns.lineplot(data=kendall_df, x="size", y="kendall", marker="o")
#plt.title('Kendall\'s W vs. sample size')
#plt.xlabel('Sample size')
#plt.ylabel('Kendall\'s W')
#pylab.savefig(os.path.join(plots_dir, 'kendall_vs_sample_size.png'), format='png')
#plt.show()

## Sample size effect on RA module identification

In [ ]:
# Plot num. of RA genes in the network vs. sample size
fig = pylab.figure(dpi=300)
sns.boxplot(data=RA_df, x='size', y='RA_genes_in_network', color='dodgerblue')
plt.title('Sample size vs number of RA genes in the network')
plt.xlabel('Sample size')
plt.ylabel('Number of RA genes in the network')
pylab.savefig(os.path.join(plots_dir, 'RA_genes_in_network_vs_sample_size.png'), format='png')
plt.show()


In [ ]:
# Plot size of RA LCC in the network vs. sample size
fig = pylab.figure(dpi=300)
sns.boxplot(data=RA_df, x='size', y='RA_lcc_size', color='dodgerblue')
plt.title('Sample size vs size of RA LCC in the gene co-expression network')
plt.xlabel('Sample size')
plt.ylabel('Number of nodes in the RA LCC')
pylab.savefig(os.path.join(plots_dir, 'RA_LCC_size_in_network_vs_sample_size.png'), format='png')
plt.show()
